# **INSTALLATION**

This code installs essential Python libraries used for natural language processing (NLP), model training, and hyperparameter tuning. It sets up the environment by updating and installing Transformers for pre-trained models, Datasets for data handling, Accelerate for efficient GPU usage, and Ray[Tune] and Optuna for automated hyperparameter optimization. The input is the command itself, and the output is a ready-to-use environment with all necessary packages installed for fine-tuning and evaluating transformer-based models like RoBERTa.

In [ ]:
!pip install transformers datasets accelerate ray[tune] optuna -U

# **IMPORT**

This code initializes the environment and prepares all necessary libraries for fine-tuning the RoBERTa model using Google Colab. Its purpose is to import essential packages for data handling, model training, evaluation, and random search optimization with Optuna, while also ensuring consistent results by setting a fixed random seed. The input consists of Python library imports and hardware checks, and the output displays whether the system is using a GPU or CPU, confirming that the environment is properly configured for model training and evaluation.

In [ ]:
# 1. SETUP AND INSTALLATION
# Run this command first in your Colab notebook:
# !pip install transformers datasets accelerate ray[tune] optuna pandas -U

import torch
import os
import numpy as np
import pandas as pd
from datasets import Dataset
from transformers import (
    AutoModelForSequenceClassification,
    AutoTokenizer,
    TrainingArguments,
    Trainer,
    set_seed
)
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
import optuna # Import optuna to use its suggestion methods for random search

# Set a consistent seed for reproducibility across runs
set_seed(42)

# Ensure GPU is available
if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f"Using GPU: {torch.cuda.get_device_name(0)}")
else:
    device = torch.device("cpu")
    print("Using CPU. For faster training, consider enabling a GPU runtime.")

Using GPU: Tesla T4


# **DATA PREPARATION**

This code prepares and processes the dataset needed for fine-tuning the RoBERTa model on mental health–related tweets. Its main purpose is to load the “Mental-Health-Twitter.csv” file, clean it by removing empty or missing text entries, and split it into training and evaluation subsets to ensure balanced model learning while staying within GPU memory limits. The input is a raw CSV file containing tweet texts and labels, and the output is a tokenized dataset in PyTorch format that has been processed through the RoBERTa tokenizer, ready for training and evaluation using the Hugging Face Trainer framework.

In [ ]:
# --- 2. DATA PREPARATION (Using your Mental-Health-Twitter.csv) ---

# Upload 'Mental-Health-Twitter.csv' to your Colab environment
# Example: from google.colab import files
#          files.upload() # Then select your file

# Load your dataset
try:
    df = pd.read_csv("Mental-Health-Twitter.csv")
    print("Dataset loaded successfully.")
except FileNotFoundError:
    print("Error: 'Mental-Health-Twitter.csv' not found. Please upload it to your Colab environment.")
    exit()

# Filter out rows where 'post_text' is NaN or empty
df = df.dropna(subset=['post_text'])
df = df[df['post_text'].str.strip() != '']

# Rename 'label' to 'labels' for Hugging Face Trainer compatibility
df = df.rename(columns={"label": "labels"})

# Split data into training and validation sets
# IMPORTANT: Keep dataset size manageable for 5GB GPU.
# 10k train / 2k eval is still quite a lot for a 5GB GPU and RoBERTa-base.
# Let's reduce it further to make sure even batch_size=16 is stable.
train_df, eval_df = train_test_split(df, test_size=0.1, stratify=df['labels'], random_state=42)
train_df = train_df.sample(n=5000, random_state=42) # REDUCED TO 5K TRAINING SAMPLES
eval_df = eval_df.sample(n=1000, random_state=42)   # REDUCED TO 1K EVALUATION SAMPLES

print(f"Using {len(train_df)} training samples and {len(eval_df)} evaluation samples.")
print(f"Train label distribution:\n{train_df['labels'].value_counts(normalize=True)}")
print(f"Eval label distribution:\n{eval_df['labels'].value_counts(normalize=True)}")

# Convert pandas DataFrames to Hugging Face Dataset objects
train_dataset = Dataset.from_pandas(train_df[['post_text', 'labels']])
eval_dataset = Dataset.from_pandas(eval_df[['post_text', 'labels']])

# Initialize Tokenizer for your specific model
MODEL_NAME = "margotwagner/roberta-psychotherapy-eval"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

def tokenize_function(examples):
    # Max length is already 128, which is good. Don't go higher.
    return tokenizer(examples["post_text"], truncation=True, padding=True, max_length=128)

# Apply tokenization
tokenized_train = train_dataset.map(tokenize_function, batched=True, remove_columns=["post_text", "__index_level_0__"])
tokenized_eval = eval_dataset.map(tokenize_function, batched=True, remove_columns=["post_text", "__index_level_0__"])

# Set format to PyTorch tensors
tokenized_train.set_format("torch", columns=['input_ids', 'attention_mask', 'labels'])
tokenized_eval.set_format("torch", columns=['input_ids', 'attention_mask', 'labels'])

Dataset loaded successfully.
Using 5000 training samples and 1000 evaluation samples.
Train label distribution:
labels
1    0.5
0    0.5
Name: proportion, dtype: float64
Eval label distribution:
labels
0    0.507
1    0.493
Name: proportion, dtype: float64


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/394 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

# **MODEL, METRICS, AND HYPERPARAMETER DEFINITION**

This code defines the model setup, evaluation metrics, and hyperparameter search space for fine-tuning the RoBERTa model. Its purpose is to initialize a new RoBERTa classifier for each tuning trial, calculate essential performance metrics such as accuracy, F1 score, precision, and recall, and specify a random search space that fits within GPU memory constraints. The input consists of parameter suggestions from the Optuna trial, while the output generates a configuration dictionary containing optimized values for learning rate, batch size, gradient accumulation steps, weight decay, and number of epochs.

In [ ]:
# --- 3. MODEL, METRICS, AND HYPERPARAMETER DEFINITION ---

# Function to initialize a fresh model for each grid search run
def model_init():
    return AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=2).to(device)

def compute_metrics(p):
    preds = np.argmax(p.predictions, axis=1)
    acc = accuracy_score(p.label_ids, preds)
    f1 = f1_score(p.label_ids, preds, average="binary")
    precision = precision_score(p.label_ids, preds, average="binary")
    recall = recall_score(p.label_ids, preds, average="binary")
    return {"accuracy": acc, "f1": f1, "precision": precision, "recall": recall}

# --- HYPERPARAMETER SEARCH SPACE FOR RANDOM SEARCH (5GB GPU CONSCIOUS) ---
def random_hp_space(trial):
    """
    This function defines the hyperparameter search space for Random Search,
    optimized for a 5GB GPU memory limit.
    """
    # 1. Learning Rate (log-uniform distribution is common for learning rates)
    learning_rate = trial.suggest_loguniform("learning_rate", 1e-5, 3e-5) # Broader range

    # 2. Batch Size (CRITICAL FOR 5GB GPU)
    # We must be very conservative here. Batch size 32 is likely too large.
    # We will mainly stick to 8 or 16, and consider gradient_accumulation_steps.
    per_device_train_batch_size = trial.suggest_categorical("per_device_train_batch_size", [4, 16, 32])

    # 3. Gradient Accumulation Steps (compensates for small batch sizes)
    # This effectively makes the 'effective' batch size larger without increasing VRAM.
    # effective_batch_size = per_device_train_batch_size * gradient_accumulation_steps
    gradient_accumulation_steps = trial.suggest_categorical("gradient_accumulation_steps", [1, 2, 3]) # Try accumulating gradients

    # 4. Weight Decay (uniform float distribution)
    weight_decay = trial.suggest_float("weight_decay", 0.01, 0.03, step=0.02)

    # 5. Number of Training Epochs (keep low for faster trials)
    num_train_epochs = trial.suggest_int("num_train_epochs", 1, 2, 3) # Max 3 epochs

    return {
        "learning_rate": learning_rate,
        "per_device_train_batch_size": per_device_train_batch_size,
        "gradient_accumulation_steps": gradient_accumulation_steps, # New HP
        "weight_decay": weight_decay,
        "num_train_epochs": num_train_epochs,
    }

# **TRAINING ARGUMENTS**

This code sets the training configuration and evaluation setup for fine-tuning the RoBERTa model using the Hugging Face Trainer API. Its purpose is to define consistent training arguments across all random search experiments, including model saving strategies, evaluation intervals, GPU optimization settings, and logging details. It also initializes the Trainer with the prepared datasets, tokenizer, and evaluation metrics to streamline the training and validation process.

The input includes tokenized datasets, model initialization, and metric computation functions, while the output produces a fully configured Trainer ready for experimentation. Additionally, the optuna_hp_objective() function defines how Optuna evaluates each hyperparameter combination, returning the F1 score as the optimization target. This setup ensures that the model is trained efficiently, monitored systematically, and optimized based on measurable performance improvements.

In [ ]:
# --- 4. TRAINING ARGUMENTS (Fixed for all runs) ---
training_args = TrainingArguments(
    output_dir="./random_search_results_mental_health_5gb", # New output directory
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    fp16=torch.cuda.is_available(), # Enable mixed precision for T4 GPU
    report_to="none",
    num_train_epochs=3, # Placeholder, will be suggested by random_hp_space
    warmup_steps=100,
    logging_dir="./logs_random_5gb", # New logging directory
    logging_steps=500,
    dataloader_num_workers=os.cpu_count() // 2 if os.cpu_count() else 0,
    # gradient_accumulation_steps will be passed directly from random_hp_space
)

# Initialize the Trainer
trainer = Trainer(
    model_init=model_init,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_eval,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
)

# --- Define the objective function for Optuna ---
def optuna_hp_objective(metrics):
    """
    Optuna objective function that returns the F1 score for maximization.
    `metrics` is the dictionary returned by trainer.evaluate().
    """
    return metrics["eval_f1"]

/tmp/ipython-input-339085150.py:19: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


config.json:   0%|          | 0.00/886 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/499M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

# **EXECUTION OF RANDOM SEARCH**

This code executes the random search optimization process using Optuna to identify the best hyperparameter combination for fine-tuning the RoBERTa model on depression detection tweets. Its main purpose is to test multiple hyperparameter configurations, such as learning rate, batch size, gradient accumulation, and epochs, while monitoring model performance through the F1 score. The process is designed for a 5GB GPU limit, using smaller batch sizes and reduced datasets to prevent memory overload and ensure smoother training. Once the trials are completed, the best-performing hyperparameters are displayed and saved for final model training.

The input for this code includes the preprocessed datasets, defined hyperparameter search space, and evaluation metrics. The output is a fine-tuned RoBERTa model trained with the optimal hyperparameter configuration, along with its final evaluation results. By balancing computational efficiency and accuracy, this step ensures that the final model achieves strong predictive performance while remaining suitable for limited GPU resources.

In [ ]:
# --- 5. EXECUTION OF RANDOM SEARCH ---
print("\n--- Starting Random Search (using Optuna backend) ---")
print(f"Optimizing for '{training_args.metric_for_best_model}' score...")

NUM_RANDOM_TRIALS = 20

print(f"Number of random trials to run: {NUM_RANDOM_TRIALS}")
print("NOTE: Batch sizes are kept low and gradient accumulation is used to manage 5GB GPU memory.")
print("      Dataset size has also been reduced for quicker iteration.")

best_trial = trainer.hyperparameter_search(
    backend="optuna",
    hp_space=random_hp_space, # Use the new random_hp_space function
    direction="maximize",
    n_trials=NUM_RANDOM_TRIALS,
    compute_objective=optuna_hp_objective,
)

print("\n--- Random Search Complete ---")
print("\nBEST HYPERPARAMETERS FOUND:")

# Extract and print the best configuration
if best_trial:
    print(best_trial)
    best_hps = best_trial.hyperparameters
    print("\nBest Hyperparameters:")
    for key, value in best_hps.items():
        print(f"  {key}: {value}")
    print(f"\nBest Metrics (on evaluation set): {best_trial.metrics}")
else:
    print("Search failed or no best trial found.")

print("\n--- Final Step: Train a model with the best hyperparameters ---")
if best_trial:
    final_training_args = TrainingArguments(
        output_dir="./final_model_mental_health_5gb_random", # New output directory
        eval_strategy="epoch",
        save_strategy="epoch",
        load_best_model_at_end=True,
        metric_for_best_model="f1",
        fp16=torch.cuda.is_available(),
        report_to="none",
        num_train_epochs=best_hps["num_train_epochs"],
        per_device_train_batch_size=best_hps["per_device_train_batch_size"],
        gradient_accumulation_steps=best_hps["gradient_accumulation_steps"], # Apply best G.A.S.
        learning_rate=best_hps["learning_rate"],
        weight_decay=best_hps["weight_decay"],
        warmup_steps=100,
        logging_dir="./final_logs_random_5gb", # New logging directory
        logging_steps=500,
        dataloader_num_workers=os.cpu_count() // 2 if os.cpu_count() else 0,
    )

    final_trainer = Trainer(
        model_init=model_init,
        args=final_training_args,
        train_dataset=tokenized_train,
        eval_dataset=tokenized_eval,
        compute_metrics=compute_metrics,
        tokenizer=tokenizer,
    )

    print("\nTraining final model with best hyperparameters from Random Search (5GB GPU config)...")
    final_trainer.train()

    print("\nFinal model training complete. Best model saved to './final_model_mental_health_5gb_random'.")
    metrics = final_trainer.evaluate()
    print(f"Evaluation metrics of the final model: {metrics}")
else:
    print("No best hyperparameters found, skipping final model training.")

[I 2025-11-08 11:23:43,341] A new study created in memory with name: no-name-7e65ec64-bc46-48a6-abf4-588f8fcc4a70



--- Starting Random Search (using Optuna backend) ---
Optimizing for 'f1' score...
Number of random trials to run: 20
NOTE: Batch sizes are kept low and gradient accumulation is used to manage 5GB GPU memory.
      Dataset size has also been reduced for quicker iteration.


/tmp/ipython-input-2811780632.py:22: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform("learning_rate", 1e-5, 3e-5) # Broader range
/tmp/ipython-input-2811780632.py:38: FutureWarning: suggest_int() got {'step'} as positional arguments but they were expected to be given as keyword arguments.
Positional arguments ['self', 'name', 'low', 'high', 'step', 'log'] in suggest_int() have been deprecated since v3.5.0. They will be replaced with the corresponding keyword arguments in v5.0.0, so please use the keyword specification instead. See https://github.com/optuna/optuna/releases/tag/v3.5.0 for details.
  num_train_epochs = trial.suggest_int("num_train_epochs", 1, 2, 3) # Max 3 epochs
/usr/local/lib/python3.12/dist-packages/optuna/distributions.py:702: UserWarning: The distribution is specified by

Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,No log,0.536950,0.725000,0.754245,0.674121,0.855984


[I 2025-11-08 11:24:36,403] Trial 0 finished with value: 0.7542448614834674 and parameters: {'learning_rate': 1.8703743417060368e-05, 'per_device_train_batch_size': 16, 'gradient_accumulation_steps': 3, 'weight_decay': 0.03, 'num_train_epochs': 1}. Best is trial 0 with value: 0.7542448614834674.
/tmp/ipython-input-2811780632.py:22: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform("learning_rate", 1e-5, 3e-5) # Broader range
/tmp/ipython-input-2811780632.py:38: FutureWarning: suggest_int() got {'step'} as positional arguments but they were expected to be given as keyword arguments.
Positional arguments ['self', 'name', 'low', 'high', 'step', 'log'] in suggest_int() have been deprecated since v3.5.0. They will be replaced with the corresponding keyword arguments in v5.0.0, so please use the k

...

[I 2025-11-08 11:54:03,375] Trial 18 pruned. 
/tmp/ipython-input-2811780632.py:22: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform("learning_rate", 1e-5, 3e-5) # Broader range
/tmp/ipython-input-2811780632.py:38: FutureWarning: suggest_int() got {'step'} as positional arguments but they were expected to be given as keyword arguments.
Positional arguments ['self', 'name', 'low', 'high', 'step', 'log'] in suggest_int() have been deprecated since v3.5.0. They will be replaced with the corresponding keyword arguments in v5.0.0, so please use the keyword specification instead. See https://github.com/optuna/optuna/releases/tag/v3.5.0 for details.
  num_train_epochs = trial.suggest_int("num_train_epochs", 1, 2, 3) # Max 3 epochs
/usr/local/lib/python3.12/dist-packages/optuna/distributions.py:702:

Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,No log,0.519463,0.724000,0.715464,0.727463,0.703854


[I 2025-11-08 11:54:36,015] Trial 19 pruned. 



--- Random Search Complete ---

BEST HYPERPARAMETERS FOUND:
BestRun(run_id='15', objective=0.875968992248062, hyperparameters={'learning_rate': 2.9730706223808927e-05, 'per_device_train_batch_size': 4, 'gradient_accumulation_steps': 1, 'weight_decay': 0.03, 'num_train_epochs': 1}, run_summary=None)

Best Hyperparameters:
  learning_rate: 2.9730706223808927e-05
  per_device_train_batch_size: 4
  gradient_accumulation_steps: 1
  weight_decay: 0.03
  num_train_epochs: 1


AttributeError: 'BestRun' object has no attribute 'metrics'